In [2]:
!pip install scispacy
!pip install spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

     |████████████████████████████████| 44 kB 1.2 MB/s 
     |████████████████████████████████| 71 kB 4.4 MB/s 
     |████████████████████████████████| 6.3 MB 20.5 MB/s 
     |████████████████████████████████| 13.5 MB 34.9 MB/s 
     |████████████████████████████████| 188 kB 43.0 MB/s 
     |████████████████████████████████| 628 kB 45.9 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 451 kB 23.4 MB/s 
     |████████████████████████████████| 10.1 MB 36.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall

In [3]:
import scispacy
import json
import spacy
from sklearn.utils import shuffle
import random

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [9]:
%ls

 269_presentation_paper.gslides
'269 Presentation - project.gslides'
'Colab Notebooks'/
'Diagram & Graphs.gslides'
'Getting started.pdf'
 journal_club_local/
 journal_club_share@
 notes1.gdoc
 Notes.gdoc
 Pitch.gslides
'Prototyping presentation (1).gslides'
'Prototyping presentation.gslides'
'RESUME-HXR work.pdf'
 seq/
'Sequential regulatory activity prediction across chromosomes with convolutional neural networks.gslides'
 SS-NER-full.json
 SS-NER.json
 SS-NER-prune.json
 teaser.mp4
'Untitled presentation.gslides'


## Data

In [6]:
types = ('LOC', 'CORONAVIRUS', 'LIVESTOCK', 'WILDLIFE', 'EVOLUTION', 'PHYSICAL_SCIENCE',
'SUBSTRATE', 'MATERIAL', 'IMMUNE_RESPONSE')

In [7]:
X = []

In [10]:
with open('./SS-NER-prune.json') as file:
    for line in file:
        d = json.loads(line)
        X.append(d)

In [41]:
test_X = X[:int(len(X)*(0.2))]
train_X = X[int(len(X)*(0.4)):]

## Spacy

### en_ner_bc5cdr

In [13]:
from spacy.util import minibatch, compounding
import random
from spacy.training.example import Example

In [36]:
bc5cdr = spacy.load("en_ner_bc5cdr_md")

In [37]:
bc5cdr.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']

In [38]:
ner = bc5cdr.get_pipe('ner')

In [30]:
for name in bc5cdr.pipe_names:
    if name != "ner":
        bc5cdr.disable_pipe(name)

In [39]:
for i in types:
    ner.add_label(i)

In [40]:
optimizer = ner.create_optimizer()

In [33]:
new_train = []
for x in train_X:
    if 'body' in x.keys() and 'entities' in x.keys():
        text = x['body']
        tmp = []
        for i in range(len(x['entities'])):
            cur = x['entities'][i]
            tmp.append((cur['start'], cur['end'], cur['type']))
        new_train.append((text, {"entities": tmp}))

In [42]:
n_iter = 1

In [ ]:
for itn in range(n_iter):
    random.shuffle(new_train)
    losses = {}
    batches = minibatch(new_train, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
            # Update the model with iterating each text
        for i in range(len(texts)):
            if len(texts[i]) > 100000:
              continue
            doc = bc5cdr.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        # Update the model
        bc5cdr.update(example, sgd=optimizer, drop=0.5, losses=losses)
        # ner.update(texts, annotations, sgd=optimizer, drop=0.35,
        #                 losses=losses)
            # doc = bc5cdr.make_doc(texts)
            # for a in annotations:
            #     example.append(Example.from_dict(doc, a))
                
            # Updating the weights
        # bc5cdr.update(example, sgd=optimizer, drop=0.35, losses=losses)
    print('Losses', losses)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Enterovirus D68-associated respiratory and neurolo..." with entities "[(990, 1002, 'EVOLUTION'), (2827, 2833, 'LOC'), (5...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Pattern of early human-to-human transmission of Wu..." with entities "[(54, 63, 'CORONAVIRUS'), (251, 262, 'CORONAVIRUS'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len

In [ ]:
bc5cdr.meta['name'] = 'model60'  # rename model
nlp.to_disk('./')

#### Evaluation

In [17]:
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

In [18]:
scorer = Scorer(bc5cdr)

In [ ]:
new_test = []
for t in test_X:
    txt = t['body']
    et = {}
    lst = []
    for e in t['entities']:
        lst.append((e['start'],e['end'],e['type']))
    et['entities'] = lst
    example = Example.from_dict(bc5cdr.make_doc(txt), et)
    new_test.append(example)

In [20]:
scores = scorer.score(new_test)

In [21]:
evaluate(bc5cdr, new_test)

{'dep_las': None,
 'dep_las_per_type': None,
 'dep_uas': None,
 'ents_f': 0.0,
 'ents_p': 0.0,
 'ents_per_type': {'CORONAVIRUS': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'EVOLUTION': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'IMMUNE_RESPONSE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LIVESTOCK': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LOC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'MATERIAL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PHYSICAL_SCIENCE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'SUBSTRATE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'WILDLIFE': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'ents_r': 0.0,
 'lemma_acc': None,
 'morph_acc': None,
 'morph_per_feat': None,
 'pos_acc': None,
 'sents_f': None,
 'sents_p': None,
 'sents_r': None,
 'tag_acc': None,
 'token_acc': 1.0,
 'token_f': 1.0,
 'token_p': 1.0,
 'token_r': 1.0}

In [ ]:
pred_y = []
for d in test_X:
    doc = bc5cdr(d)
    tmp = []
    for e in doc.ents:
        tmp.append(e.text)
    pred_y.append(tmp)

In [ ]:
def f1(y_true, y_pred):
    i = list((Counter(y_true) & Counter(y_pred)).elements())
    # i = set(y_true).intersection(y_pred)
    print(len(i))
    recall = len(i) / len(y_true)
    precision = len(i) / len(y_pred)
    print('recall', recall)
    print('precision', precision)
    if recall + precision == 0:
        return 0
    else:
        return 2 * (precision * recall) / (precision + recall)


In [ ]:
def avgf1(true_y, pred_y):
    f_list = []
    for t, p in zip(true_y, pred_y):
        f_list.append(f1(t, p))
    return sum(f_list)/len(f_list)